In [ ]:
import numpy as np 
import pandas as pd 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
from pathlib import Path

import os

#Source: Stockholm International Peace Research Institute, SIPRI Arms Transfers Database
# IMPORTER/EXPORTER TIV TABLES: http://armstrade.sipri.org/armstrade/page/toplist.php

data_folder = Path("input/Sipri-dataset/")  
file_to_open = data_folder / "TIV-Export-Top-50-2000-2019.csv"  
print(open(file_to_open).readlines())


In [ ]:
df = pd.read_csv(file_to_open, skiprows = 8)
df_orig = df.copy()
df.head()

In [ ]:
print(list(df))

In [ ]:
df = df.drop(columns = ['Rank 2000-2019','Rank 1999-2018'])
df.head()

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = df['Supplier'],
    locationmode =  'country names',  #'ISO-3'
    z = df['2000-2019'],
    colorscale = 'Reds',
    marker_line_color = 'black',
    marker_line_width = 0.5,
))

fig.update_layout(
    title_text = 'TIV of arms exports from the top 50 largest exporters, 2000-2019',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
        projection_type = 'equirectangular'
    )
)

#locationmode: Determines the set of locations used to match entries in `locations` to regions on the map
#Parent: data[type=choropleth], Type: enumerated , one of ( "ISO-3" | "USA-states" | "country names" | "geojson-id" )
#Default: "ISO-3"


In [ ]:
df = df.drop(columns = ['2000-2019'])
df.head(7)

In [ ]:
#initialize Dataframe
df_sort = pd.DataFrame()

#Splitting and transpose DataFrame, so we got all biggest exporter countries separated
#Plotly Express operates on "tidy" or "long" data rather than "wide" data, change DataFrame to columns:
#Year, Country Value

def split_DataFrame(df,start, stop):
    df_1country = df[start:stop]
    df_1country = df_1country.T  #transpose
    df_1country[stop] = df['Supplier'].loc[start]
    df_1country.rename_axis("Year", axis = 'index', inplace=True)   #rename index,
    df_1country.columns = ['Value', 'Country']
    #df_1country = df_1country.drop(["Supplier"], axis = 0)   #drops 1st row
    df_1country = df_1country.iloc[1:, ]   #delete first row using iloc selector
    return df_1country

for i in range (6):
    df_temp = split_DataFrame(df, i, i+1)
    df_sort = df_sort.append(df_temp)

df_sort.tail()

In [ ]:
df_tidy = df_sort.sort_values(by = ['Year', 'Country'])
df_tidy.head(15)

In [ ]:
df_tidy.tail(7)

Most Plotly Express functions accept a color argument which automatically assigns data values to continuous color if the data is numeric. If the data contains strings, the color will automatically be considered discrete (also known as categorical or qualitative). This means that numeric strings must be parsed to be used for continuous color, and conversely, numbers used as category codes must be converted to strings.
Source: https://plot.ly/python/colorscales/

In [ ]:
df_tidy.dtypes

In [ ]:
#https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas
df_tidy['Value'] = pd.to_numeric(df_tidy['Value'], downcast ='signed') # convert 'Value' to int

In [ ]:
df_tidy.dtypes

In [ ]:
print(list(df_tidy))

In [ ]:
print(df_tidy.index)

In [ ]:
fig = px.choropleth(df_tidy, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Value", 
                    #color_continuous_scale=px.colors.sequential.Plasma,
                    color_continuous_scale="Viridis",
                    range_color=(0, 12000),
                    labels={'Value':'TIV of arms exports'},
                    hover_name="Value", 
                    animation_frame= df_tidy.index   #animate over Years
                   )

fig.update_layout(
    title_text = 'Weapon Exporters',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()


In [ ]:
fig = px.bar(df_tidy, y="Country", x="Value", color="Value", range_color=(0, 12000), labels=
             {'Value':'TIV of arms exports'},orientation="h", color_continuous_scale=
             'Viridis', animation_frame= df_tidy.index )

fig.update_layout(
    title_text = 'Weapon Exporters',
    title_x = 0.5,
    )

fig.update_xaxes(range=[0, 12000])
fig.show()